In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate, SimpleRNN, Reshape
from sklearn.preprocessing import MinMaxScaler


x_values = np.linspace(-5, 5, 1000)
y_values = 7 * np.sin(2.5 * np.cos(x_values))
z_values = (x_values - 2) ** 2 * (1 - y_values**2)

# Data normalizing for lowering the value of MSE/MAE due to function value scaling
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_z = MinMaxScaler()

x = scaler_x.fit_transform(x_values.reshape(-1, 1)).flatten()
y = scaler_y.fit_transform(y_values.reshape(-1, 1)).flatten()
z = scaler_z.fit_transform(z_values.reshape(-1, 1)).flatten()


def model_testing(model):
    model.compile(optimizer='adam', loss='mse')
    model.fit(x, z, epochs=20, batch_size=100)
    z_pred = model.predict(x)

    plt.plot(x, z, label='Actual')
    plt.plot(x, z_pred, label='Predicted')
    plt.legend()
    plt.show()


def feed_forward_creation(layers, neurons):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(1,)))

    for i in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, name='output'))
    return model


def cascade_forward_creation(layers, neurons):
    inputLayer = Input(shape=(1,), name='input')
    current = Dense(neurons, activation='relu', input_shape=(1,))(inputLayer)

    for i in range(layers-1):
        concatenatedLayer = Concatenate()([inputLayer, current])
        current = Dense(neurons, activation='relu', input_shape=(1,))(concatenatedLayer)

    outputLayer = Dense(1, name='output')(current)
    model = Model(inputs=inputLayer, outputs=outputLayer)

    return model


def elman_creation(layers, neurons):
    model = Sequential()
    model.add(Reshape((1, 1), input_shape=(1,), name='input_reshape'))
    model.add(SimpleRNN(neurons, return_sequences=True, activation='relu', 
                        input_shape=(1,)))

    for i in range(layers - 1):
        model.add(SimpleRNN(neurons, return_sequences=True, activation='relu'))

    model.add(Dense(1, name='output'))
    model.add(Reshape((1,), input_shape=(1, 1), name='output_reshape'))

    return model


f1 = feed_forward_creation(1, 10)
f2 = feed_forward_creation(1, 20)
c1 = cascade_forward_creation(1, 20)
c2 = cascade_forward_creation(2, 10)
e1 = elman_creation(1, 15)
e2 = elman_creation(3, 5)

model_testing(f1)
model_testing(f2)
model_testing(c1)
model_testing(c2)
model_testing(e1)
model_testing(e2)